In [1]:
import tensorflow as tf
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
#load pickle file
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save

print ('Training set: ', train_dataset.shape, train_labels.shape)
print ('Validation set: ', valid_dataset.shape, valid_labels.shape)
print ('Testing set: ', test_dataset.shape, test_labels.shape)

('Training set: ', (200000, 28, 28), (200000,))
('Validation set: ', (10000, 28, 28), (10000,))
('Testing set: ', (10000, 28, 28), (10000,))


In [3]:
image_size = 28
num_labels = 10

# reformat the data 
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # labels -> one-hot encoding 
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print ('Training set: ', train_dataset.shape, train_labels.shape)
print ('Validation set: ', valid_dataset.shape, valid_labels.shape)
print ('Testing set: ', test_dataset.shape, test_labels.shape)

# output labels accuracy  
def accuracy(predictions, labels):
    return(100. * np.sum(np.argmax(predictions, 1)==np.argmax(labels, 1))
          / predictions.shape[0])

('Training set: ', (200000, 784), (200000, 10))
('Validation set: ', (10000, 784), (10000, 10))
('Testing set: ', (10000, 784), (10000, 10))


In [4]:
# create default graph - Gradient Descent training
train_subset = 10000

graph = tf.Graph()
with graph.as_default(): 
    # input data - load train, valid and test data
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    # training params
    logits = tf.matmul(tf_train_dataset, weights) + biases 
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for training, validation and test
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 2000

# run the model
with tf.Session(graph=graph) as sess:
    # initialize params
    tf.global_variables_initializer().run()
    print ('Initialized')
    
    for step in range(num_steps):
        _, l, predictions = sess.run([optimizer, loss, train_predictions])
        if (step % 100 == 0):
            print ('Loss at step %d : %f' % (step, l))
            print ('Train accuracy : %.2f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print ('Validation accuracy : %.2f%%' % accuracy(
                valid_predictions.eval(), valid_labels))
    print ('Test accuracy : %.2f%%' % accuracy(test_predictions.eval(),
                                                test_labels))

Initialized
Loss at step 0 : 20.520744
Train accuracy : 7.19%
Validation accuracy : 9.35%
Loss at step 100 : 2.291584
Train accuracy : 72.10%
Validation accuracy : 70.29%
Loss at step 200 : 1.851238
Train accuracy : 74.93%
Validation accuracy : 72.66%
Loss at step 300 : 1.614115
Train accuracy : 76.20%
Validation accuracy : 73.42%
Loss at step 400 : 1.452840
Train accuracy : 77.13%
Validation accuracy : 73.98%
Loss at step 500 : 1.333231
Train accuracy : 77.69%
Validation accuracy : 74.21%
Loss at step 600 : 1.239667
Train accuracy : 78.12%
Validation accuracy : 74.48%
Loss at step 700 : 1.163460
Train accuracy : 78.67%
Validation accuracy : 74.60%
Loss at step 800 : 1.099659
Train accuracy : 79.08%
Validation accuracy : 74.76%
Loss at step 900 : 1.045239
Train accuracy : 79.56%
Validation accuracy : 75.01%
Loss at step 1000 : 0.998148
Train accuracy : 79.81%
Validation accuracy : 75.15%
Loss at step 1100 : 0.956907
Train accuracy : 80.19%
Validation accuracy : 75.38%
Loss at step 1200

In [5]:
# stochastic gradient descent
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_predictions = tf.nn.softmax(logits)
  valid_predictions = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3000

# run the model
with tf.Session(graph=graph) as sess:
    # initialize params
    tf.global_variables_initializer().run()
    print ('Initialized')
    
    for step in range(num_steps):
        # generate offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #print batch_data.shape
        # create dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        #print feed_dict
        _, l, predictions = sess.run(
          [optimizer, loss, train_predictions], feed_dict=feed_dict)
        if (step % 300 == 0):
            print ('Minibatch Loss at step %d : %f' % (step, l))
            print ('Minibatch accuracy : %.2f%%' % accuracy(predictions, batch_labels))
            print ('Validation accuracy : %.2f%%' % accuracy(
                valid_predictions.eval(), valid_labels))
    print ('Test accuracy : %.2f%%' % accuracy(test_predictions.eval(),
                                                test_labels))

Initialized
Minibatch Loss at step 0 : 16.057673
Minibatch accuracy : 11.72%
Validation accuracy : 16.02%
Minibatch Loss at step 300 : 2.133955
Minibatch accuracy : 72.66%
Validation accuracy : 73.93%
Minibatch Loss at step 600 : 1.716676
Minibatch accuracy : 72.66%
Validation accuracy : 76.03%
Minibatch Loss at step 900 : 1.414912
Minibatch accuracy : 71.09%
Validation accuracy : 76.14%
Minibatch Loss at step 1200 : 1.510216
Minibatch accuracy : 72.66%
Validation accuracy : 76.68%
Minibatch Loss at step 1500 : 0.858888
Minibatch accuracy : 84.38%
Validation accuracy : 77.25%
Minibatch Loss at step 1800 : 1.255108
Minibatch accuracy : 75.78%
Validation accuracy : 77.63%
Minibatch Loss at step 2100 : 1.169979
Minibatch accuracy : 78.12%
Validation accuracy : 78.08%
Minibatch Loss at step 2400 : 1.342642
Minibatch accuracy : 69.53%
Validation accuracy : 77.97%
Minibatch Loss at step 2700 : 0.706312
Minibatch accuracy : 85.94%
Validation accuracy : 78.13%
Test accuracy : 85.47%


In [6]:
# 1-layer MLP
n_hidden1 = 1024 # number of neurons in hidden layer 1

# mlp model
def single_layer_perceptron(x, weights, biases):
    # Hidden layer-1 with relu
    layer1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer1 = tf.nn.relu(layer1)
    # output layer
    out_layer = tf.matmul(layer1, weights['out']) + biases['out']
    return out_layer

# stochastic gradient descent
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # declare weights and biases for layer1 and output layer
  weights = {
    'w1' : tf.Variable(tf.random_normal([image_size * image_size, n_hidden1])),
    'out': tf.Variable(tf.random_normal([n_hidden1, num_labels]))
  }
  biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden1])),
    'out': tf.Variable(tf.random_normal([num_labels]))
  }
  
  # Training computation.
  logits = single_layer_perceptron(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_predictions = tf.nn.softmax(logits)
  valid_predictions = tf.nn.softmax(single_layer_perceptron(
      tf_valid_dataset, weights, biases))
  test_predictions = tf.nn.softmax(single_layer_perceptron(
      tf_test_dataset, weights, biases))

In [7]:
num_steps = 1000

# run the model
with tf.Session(graph=graph) as sess:
    # initialize params
    tf.global_variables_initializer().run()
    print ('Initialized')
    
    for step in range(num_steps):
        # generate offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #print batch_data.shape
        # create dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        #print feed_dict
        _, l, predictions = sess.run(
          [optimizer, loss, train_predictions], feed_dict=feed_dict)
        if (step % 50 == 0):
            print ('Minibatch Loss at step %d : %f' % (step, l))
            print ('Minibatch accuracy : %.2f%%' % accuracy(predictions, batch_labels))
            print ('Validation accuracy : %.2f%%' % accuracy(
                 valid_predictions.eval(), valid_labels))
    print ('Test accuracy : %.2f%%' % accuracy(test_predictions.eval(),test_labels))

Initialized
Minibatch Loss at step 0 : 380.545898
Minibatch accuracy : 7.03%
Validation accuracy : 6.89%
Minibatch Loss at step 50 : 76.620308
Minibatch accuracy : 64.06%
Validation accuracy : 64.38%
Minibatch Loss at step 100 : 60.553776
Minibatch accuracy : 67.19%
Validation accuracy : 70.64%
Minibatch Loss at step 150 : 42.748520
Minibatch accuracy : 75.78%
Validation accuracy : 72.01%
Minibatch Loss at step 200 : 45.538498
Minibatch accuracy : 74.22%
Validation accuracy : 74.03%
Minibatch Loss at step 250 : 50.945557
Minibatch accuracy : 71.09%
Validation accuracy : 74.87%
Minibatch Loss at step 300 : 51.890739
Minibatch accuracy : 75.00%
Validation accuracy : 75.30%
Minibatch Loss at step 350 : 36.568020
Minibatch accuracy : 78.91%
Validation accuracy : 75.92%
Minibatch Loss at step 400 : 29.178909
Minibatch accuracy : 80.47%
Validation accuracy : 75.83%
Minibatch Loss at step 450 : 43.751610
Minibatch accuracy : 75.00%
Validation accuracy : 76.38%
Minibatch Loss at step 500 : 34.